# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234468543839                   -0.50    7.0
  2   -7.249928195512       -1.81       -1.40    1.0
  3   -7.250944609845       -2.99       -1.79    4.0
  4   -7.250971869290       -4.56       -1.88    2.0
  5   -7.251323469472       -3.45       -2.55    2.0
  6   -7.251338053750       -4.84       -3.15    2.0
  7   -7.251338777875       -6.14       -3.84    2.0
  8   -7.251338793906       -7.80       -4.14    3.0
  9   -7.251338798477       -8.34       -4.77    1.0
 10   -7.251338798686       -9.68       -5.32    2.0
 11   -7.251338798701      -10.83       -5.81    3.0
 12   -7.251338798704      -11.47       -6.63    3.0
 13   -7.251338798705      -13.30       -7.15    2.0
 14   -7.251338798705      -14.45       -7.53    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07253304232137124
[ Info: Arnoldi iteration step 2: normres = 0.4209952619330018
[ Info: Arnoldi iteration step 3: normres = 0.9764982519798023
[ Info: Arnoldi iteration step 4: normres = 0.30630275775698707
[ Info: Arnoldi iteration step 5: normres = 0.4442226634871382
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (4.41e-02, 5.03e-02, 3.28e-01, 2.91e-01, 2.63e-02)
[ Info: Arnoldi iteration step 6: normres = 0.33561805204346534
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.35e-02, 2.02e-01, 2.39e-01, 8.85e-02, 7.58e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07336007037857882
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (4.91e-04, 1.72e-02, 6.81e-03, 2.68e-02, 5.42e-02)
[ Info: Arnoldi iteration step 8: normres = 0.12008079792618427
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.53e